# Problem 3.2

**Integrated Energy Grids**

**Problem 3.2**

In [1]:
import numpy as np
import numpy.linalg

List of nodes and links

In [2]:
nodes=[0,1,2,3,4]
links=[(0,1), (1,2), (1,3), (1,4), (2,4)]

Degree of each node and average degree of the network

$k_1$= 1, $k_1$=4, $k_2$=2, $k_3$=1, $k_4$=2

Average degree of the network

In [3]:
k=(1+4+2+1+2)/5
print(k)

2.0


**Degree matrix**

In [4]:
D = np.zeros((len(nodes), len(nodes)))

for node in nodes:
    D[node, node] = sum([1 if node in link else 0 for link in links])
D

array([[1., 0., 0., 0., 0.],
       [0., 4., 0., 0., 0.],
       [0., 0., 2., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 2.]])

**Adjacency matrix**

In [5]:
A = np.zeros((len(nodes), len(nodes)))

for node_a, node_b in links:
    A[node_a, node_b] = 1
    A[node_b, node_a] = 1

A

array([[0., 1., 0., 0., 0.],
       [1., 0., 1., 1., 1.],
       [0., 1., 0., 0., 1.],
       [0., 1., 0., 0., 0.],
       [0., 1., 1., 0., 0.]])

**Incidence matrix**

In [6]:
K = np.zeros((len(nodes),len(links)))

for i, (node_a, node_b) in enumerate(links):
    K[node_a,i] = 1
    K[node_b,i] = -1
    
K

array([[ 1.,  0.,  0.,  0.,  0.],
       [-1.,  1.,  1.,  1.,  0.],
       [ 0., -1.,  0.,  0.,  1.],
       [ 0.,  0., -1.,  0.,  0.],
       [ 0.,  0.,  0., -1., -1.]])

**Laplacian matrix**

a) using the adjacency matrix

In [7]:
L = D - A
L

array([[ 1., -1.,  0.,  0.,  0.],
       [-1.,  4., -1., -1., -1.],
       [ 0., -1.,  2.,  0., -1.],
       [ 0., -1.,  0.,  1.,  0.],
       [ 0., -1., -1.,  0.,  2.]])

b) using the incidence matrix

In [8]:
L=K.dot(K.T)
L

array([[ 1., -1.,  0.,  0.,  0.],
       [-1.,  4., -1., -1., -1.],
       [ 0., -1.,  2.,  0., -1.],
       [ 0., -1.,  0.,  1.,  0.],
       [ 0., -1., -1.,  0.,  2.]])

**Power flow analysis**

We know the power injection pattern for the nodes $P_i$ and we want to determine the power flows $f_l$ in the lines of the network



The reactance in every link is $x_l$=1 so $L_{ij}=\sum_{l} K_{il} \frac{1}{x_l}K_{lj}=\sum_{l} K_{il} K_{lj}$

First, we calculate the voltage angles
$P_i=\sum L_{i,j} \theta_j$

In [9]:
power_injection=[-200, 500, 600, -800, -100]

The node 0 (Germany) is the slack bus, so $\theta_0$=0.

In [10]:
theta = np.r_[0, np.linalg.solve(L[1:,1:], power_injection[1:])]
theta

array([   0.        ,  200.        ,  566.66666667, -600.        ,
        333.33333333])

Second, we use the voltage angles to compute the flows.
$f_l=\frac{1}{x_l}\sum_{j}K_{lj}\theta_j$

In [11]:
flows = K.T.dot(theta)
flows

array([-200.        , -366.66666667,  800.        , -133.33333333,
        233.33333333])

**Additional discussion on the inversion of the Laplacian matrix**

The Laplacian matrix is not invertible because it has one zero eigenvalue with eigenvector (1,1,1,1,1). 

We can only invert a matrix if it is non-singular, i.e., determinant should not be zero. 

You can also check that the rank of the matrix is 4 but the matrix has 5 columns, so the determinant is zero and we can not find the inverse


In [12]:
np.linalg.matrix_rank(L)

4

The problem is that np.linalg.inv(L) should give an error but it does not. The reason behind is the method to calculate the inverse of a matrix in numpy, as discussed here https://stackoverflow.com/questions/41841509/numpy-inverts-a-non-invertible-matrix
